# RecDP LLM - Downsize public dataset

This notebook shows how to use RecDP tools and pipeline to downsize a public fine tuning dataset and calculate kinds of evaluation scores.

# Get Started

## 1. Install pyrecdp and dependencies

In [ ]:
! DEBIAN_FRONTEND=noninteractive apt-get install -q -y openjdk-8-jre
# ! pip install recdp
! pip install 'git+https://github.com/intel/e2eAIOK.git#egg=pyrecdp&subdirectory=RecDP'

## 2. Prepare your data

In [2]:
!mkdir -p content/test_data
!cp ../../../tests/data/alpaca/alpaca_data_50.jsonl /content/test_data
!cp ../../../tests/data/dolly/dolly_sample_50.parquet /content/test_data
!cp ../../../tests/data/openorca/openorca_sample_50.parquet /content/test_data

In [ ]:
from pyrecdp.LLM import TextPipeline, ResumableTextPipeline
from pyrecdp.primitives.operations import *
import os

alpaca_prompt = ResumableTextPipeline()
alpaca_prompt.enable_statistics()
out_dir = "ResumableTextPipeline_output-alpaca"
ops = [
    JsonlReader("/content/test_data/alpaca_data_50.jsonl"),
    TextPrompt(dataset_name="alpaca", prompt_name="causal_llm_1"),
    ParquetWriter(out_dir)
]
alpaca_prompt.add_operations(ops)
ret = alpaca_prompt.execute()
del alpaca_prompt

openorca_prompt = ResumableTextPipeline()
openorca_prompt.enable_statistics()
out_dir = "ResumableTextPipeline_output-openorca"
ops = [
    ParquetReader("/content/test_data/openorca_sample_50.parquet"),
    TextPrompt(dataset_name="openorca", prompt_name="causal_llm_1"),
    ParquetWriter(out_dir)
]
openorca_prompt.add_operations(ops)
ret = openorca_prompt.execute()
del openorca_prompt

dolly_prompt = ResumableTextPipeline()
dolly_prompt.enable_statistics()
out_dir = "ResumableTextPipeline_output-dolly"
ops = [
    ParquetReader("/content/test_data/dolly_sample_50.parquet"),
    TextPrompt(dataset_name="dolly", prompt_name="causal_llm_1"),
    ParquetWriter(out_dir)
]
dolly_prompt.add_operations(ops)
ret = dolly_prompt.execute()
del dolly_prompt

2023-12-04 03:09:07.594 | WARNING  | pyrecdp.LLM.TextPipeline:enable_statistics:219 - Enabling this option will result in a decrease in execution speed
recdp_promptsource: /host/mnt/DP_disk1/code/recllm/e2eAIOK/RecDP/pyrecdp/promptsource
promptsource_templates_path: /opt/conda/envs/chatbot-finetuning/lib/python3.9/site-packages/promptsource/templates
recdp_promptsource: /host/mnt/DP_disk1/code/recllm/e2eAIOK/RecDP/pyrecdp/promptsource
promptsource_templates_path: /opt/conda/envs/chatbot-finetuning/lib/python3.9/site-packages/promptsource/templates
[DatasetReader, PerfileSourcedJsonlReader, TextPrompt, PerfileParquetWriter]
init ray with total mem of 162212234035


IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
2023-12-04 03:11:13,934	WARNING services.py:1889 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 67104768 bytes available. This will harm performance! You may be able to free up space by deleting files in /dev/shm. If you are inside a Docker container, you can increase /dev/shm size by passing '--shm-size=10.24gb' to 'docker run' (or add it to the run_options list in a Ray cluster config). Make sure to set this to more than 30% of available RAM.
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
2023-12-04 03:11:54,463	INFO worker.py:1642 -- Started a local Ray instance.
2023-12-04 03:11:57,227	INFO rea

Running 0:   0%|          | 0/1 [00:00<?, ?it/s]

{'instruction': 'Give three tips for staying healthy.', 'input': '', 'output': '1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. \n2. Exercise regularly to keep your body active and strong. \n3. Get enough sleep and maintain a consistent sleep schedule.'}
{'instruction': 'What are the three primary colors?', 'input': '', 'output': 'The three primary colors are red, blue, and yellow.'}
{'instruction': 'Describe the structure of an atom.', 'input': '', 'output': 'An atom is made up of a nucleus, which contains protons and neutrons, surrounded by electrons that travel in orbits around the nucleus. The protons and neutrons have a positive charge, while the electrons have a negative charge, resulting in an overall neutral atom. The number of each particle determines the atomic number and the type of atom.'}
{'instruction': 'How can we reduce air pollution?', 'input': '', 'output': 'There are a number of ways to reduce air pollution, such as shifting to renewab

2023-12-04 03:11:58,479	INFO read_api.py:406 -- To satisfy the requested parallelism of 144, each read task output is split into 144 smaller blocks.
2023-12-04 03:11:58,495	INFO streaming_executor.py:93 -- Executing DAG InputDataBuffer[Input] -> TaskPoolMapOperator[ReadJSON->SplitBlocks(144)] -> TaskPoolMapOperator[Map(<lambda>)->Map(<lambda>)->MapBatches(<lambda>)]
2023-12-04 03:11:58,499	INFO streaming_executor.py:94 -- Execution config: ExecutionOptions(resource_limits=ExecutionResources(cpu=None, gpu=None, object_store_memory=None), locality_with_output=False, preserve_order=False, actor_locality_enabled=True, verbose_progress=False)
2023-12-04 03:11:58,500	INFO streaming_executor.py:96 -- Tip: For detailed progress reporting, run `ray.data.DataContext.get_current().execution_options.verbose_progress = True`


...

(Map(<lambda>)->Map(<lambda>)->MapBatches(<lambda>)->Map(<lambda>)->Write pid=64485) /opt/conda/envs/chatbot-finetuning/lib/python3.9/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
(Map(<lambda>)->Map(<lambda>)->MapBatches(<lambda>)->Map(<lambda>)->Write pid=64485)   warnings.warn("Setuptools is replacing distutils.")
(Map(<lambda>)->Map(<lambda>)->MapBatches(<lambda>)->Map(<lambda>)->Write pid=64485) Skipping writing empty dataset with UUID ed229cdea4a4496da3e49f5a6c3d44e3 at ResumableTextPipeline_output-dolly/dolly_sample_50.parquet
(Map(<lambda>)->Map(<lambda>)->MapBatches(<lambda>)->Map(<lambda>)->Write pid=64424) /opt/conda/envs/chatbot-finetuning/lib/python3.9/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils. [repeated 71x across cluster]
(Map(<lambda>)->Map(<lambda>)->MapBatches(<lambda>)->Map(<lambda>)->Write pid=64424)   warnings.warn("Setuptools is replacing distutils.") [repeated 71x a

2023-12-04 03:18:40.686 | INFO     | pyrecdp.LLM.TextPipeline:op_summary:285 - TextPrompt: A total of 0 rows of data were processed, using 0 seconds, with 0 rows modified or removed, 0 rows of data remaining.
2023-12-04 03:18:40.694 | INFO     | pyrecdp.LLM.TextPipeline:op_summary:285 - PerfileParquetWriter: A total of 0 rows of data were processed, using 0 seconds, with 0 rows modified or removed, 0 rows of data remaining.



ResumableTextPipeline, current on dolly_sample_50.parquet: 100%|██████████| 1/1 [00:18<00:00, 18.90s/it]

2023-12-04 03:18:40.705 | INFO     | pyrecdp.LLM.TextPipeline:execute:425 - Completed! ResumableTextPipeline will not return dataset, please check ResumableTextPipeline_output-dolly for verification.



(Map(<lambda>)->Map(<lambda>)->MapBatches(<lambda>)->Map(<lambda>)->Write pid=64449) Skipping writing empty dataset with UUID ed229cdea4a4496da3e49f5a6c3d44e3 at ResumableTextPipeline_output-dolly/dolly_sample_50.parquet [repeated 86x across cluster]


## 3. Downsize dataset

### 3.1 Pipeline on Spark

In [1]:
from pyrecdp.LLM import TextPipeline, ResumableTextPipeline
from pyrecdp.primitives.operations import *
import os
alpaca_pipeline = ResumableTextPipeline()
alpaca_pipeline.enable_statistics()
out_dir = "ResumableTextPipeline_output_alpaca_result"
ops = [
    JsonlReader("/content/test_data/alpaca_data_50.jsonl"),
    TextPrompt(dataset_name="alpaca", prompt_name="causal_llm_1"),
    RandomSelect(fraction=0.3),
    TextToxicity(huggingface_config_path="/root/.cache/huggingface/hub/models--xlm-roberta-base"),
    TextDiversityIndicate(out_dir=out_dir, language="en", first_sent=False),
    TextQualityScorer(model="gpt3"),
    RougeScoreDedup(max_ratio=0.7, batch_size=10,score_store_path=os.path.join(out_dir,'RougeScorefiltered.parquet')),
    ParquetWriter(out_dir)
]
alpaca_pipeline.add_operations(ops)
ret = alpaca_pipeline.execute()
del alpaca_pipeline

JAVA_HOME is not set, use default value of /usr/lib/jvm/java-8-openjdk-amd64/


/opt/conda/envs/chatbot-finetuning/lib/python3.9/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


['psutil', 'tqdm', 'pyyaml', 'pandas', 'pyarrow', 'transformers', 'graphviz', 'requests', 'distro', 'pyspark==3.4.0', 'matplotlib', 'datasketch==1.5.9', 'ftfy==6.1.1', 'jsonlines==3.1.0', 'networkit==10.1', 'nltk==3.8.1', 'regex==2023.6.3', 'scipy==1.10.1', 'typer>=0.6.1', 'phonenumbers', 'fasttext==0.9.2', 'wget==3.2', 'alt-profanity-check==1.3.0', 'huggingface-hub', 'loguru==0.7.2', 'tabulate==0.9.0', 'sentencepiece', 'selectolax', 'spacy', 'torch', 'Faker', 'ray==2.7.1', 'loguru', 'detoxify', 'emoji==2.2.0', 'kenlm', 'rouge-score']


Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


Requirement already satisfied: alt-profanity-check==1.3.0 in /opt/conda/envs/chatbot-finetuning/lib/python3.9/site-packages (1.3.0)

...


0it [00:00, ?it/s]

generate_connected_components => duplicates took 0.03802505135536194 sec


2023-12-04 23:47:53.768 | INFO     | pyrecdp.LLM.TextPipeline:op_summary:288 - TextPrompt: A total of 0 rows of data were processed, using 0 seconds, with 0 rows modified or removed, 0 rows of data remaining.
2023-12-04 23:47:53.771 | INFO     | pyrecdp.LLM.TextPipeline:op_summary:288 - RandomSelect: A total of 0 rows of data were processed, using 0 seconds, with 0 rows modified or removed, 0 rows of data remaining.
2023-12-04 23:47:53.774 | INFO     | pyrecdp.LLM.TextPipeline:op_summary:284 - TextToxicity: A total of 11 rows of data were processed, using 69.80172777175903 seconds, Get max toxicity 0.0010136101627722383, Get min toxicity 0.0001716656406642869, Get average toxicity 0.00036636507800060576,Get the std of toxicity 0.0002271174606949808
2023-12-04 23:47:53.781 | INFO     | pyrecdp.LLM.TextPipeline:op_summary:284 - TextDiversityIndicate: A total of 11 rows of data were processed, using 15.65165090560913 seconds, Get max diversity types 11, Get average diversity types 1.625,G

ResumableTextPipeline, current on alpaca_data_50.jsonl: 100%|██████████| 1/1 [01:47<00:00, 107.56s/it]

2023-12-04 23:47:53.796 | INFO     | pyrecdp.LLM.TextPipeline:execute:429 - Completed! ResumableTextPipeline will not return dataset, please check ResumableTextPipeline_output_alpaca_result for verification.


In [4]:
from pyrecdp.LLM import TextPipeline, ResumableTextPipeline
from pyrecdp.primitives.operations import *
import os
openorca_pipeline = ResumableTextPipeline()
openorca_pipeline.enable_statistics()
out_dir = "ResumableTextPipeline_output_openorca_result"
ops = [
    ParquetReader("/content/test_data/openorca_sample_50.parquet"),
    TextPrompt(dataset_name="openorca", prompt_name="causal_llm_1"),
    RandomSelect(fraction=0.3),
    # TextToxicity(huggingface_config_path="/root/.cache/huggingface/hub/models--xlm-roberta-base"),
    TextDiversityIndicate(out_dir=out_dir, language="en", first_sent=False),
    TextQualityScorer(model="gpt3"),
    RougeScoreDedup(max_ratio=0.7, batch_size=10,score_store_path=os.path.join(out_dir,'RougeScorefiltered.parquet')),
    ParquetWriter(out_dir)
]
openorca_pipeline.add_operations(ops)
ret = openorca_pipeline.execute()
del openorca_pipeline

2023-12-05 00:12:03.708 | WARNING  | pyrecdp.LLM.TextPipeline:enable_statistics:222 - Enabling this option will result in a decrease in execution speed
[DatasetReader, PerfileSourcedParquetReader, TextPrompt, RandomSelect, TextDiversityIndicate, TextQualityScorer, RougeScoreDedup, PerfileParquetWriter]
Will assign 36 cores and 206263 M memory for spark
per core memory size is 5.595 GB and shuffle_disk maximum capacity is 8589934592.000 GB
execute with spark for global tasks started ...
DatasetReader
2023-12-05 00:12:04.078 | INFO     | pyrecdp.LLM.TextPipeline:op_summary:288 - DatasetReader: A total of 0 rows of data were processed, using 0 seconds, with 0 rows modified or removed, 0 rows of data remaining.
execute with spark for global tasks took 0.0032089054584503174 sec
PerfileSourcedParquetReader


ResumableTextPipeline, current on openorca_sample_50.parquet:   0%|          | 0/1 [00:00<?, ?it/s]

openorca_sample_50.parquet
TextPrompt


RandomSelect
TextDiversityIndicate
statistics_decorator spark


/opt/conda/envs/chatbot-finetuning/lib/python3.9/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


...


0it [00:00, ?it/s]

generate_connected_components => duplicates took 0.015356998890638351 sec


2023-12-05 00:12:31.702 | INFO     | pyrecdp.LLM.TextPipeline:op_summary:288 - TextPrompt: A total of 0 rows of data were processed, using 0 seconds, with 0 rows modified or removed, 0 rows of data remaining.
2023-12-05 00:12:31.705 | INFO     | pyrecdp.LLM.TextPipeline:op_summary:288 - RandomSelect: A total of 0 rows of data were processed, using 0 seconds, with 0 rows modified or removed, 0 rows of data remaining.
2023-12-05 00:12:31.713 | INFO     | pyrecdp.LLM.TextPipeline:op_summary:284 - TextDiversityIndicate: A total of 11 rows of data were processed, using 14.471717834472656 seconds, Get max diversity types 11, Get average diversity types 1.826086956521739,Get the std of diversity types 2.328717436956612
2023-12-05 00:12:31.715 | INFO     | pyrecdp.LLM.TextPipeline:op_summary:284 - TextQualityScorer: A total of 11 rows of data were processed, using 1.4916315078735352 seconds, Get average quality score 0.9467587080940774
2023-12-05 00:12:31.718 | INFO     | pyrecdp.LLM.TextPipel

ResumableTextPipeline, current on openorca_sample_50.parquet: 100%|██████████| 1/1 [00:27<00:00, 27.38s/it]

2023-12-05 00:12:31.724 | INFO     | pyrecdp.LLM.TextPipeline:execute:429 - Completed! ResumableTextPipeline will not return dataset, please check ResumableTextPipeline_output_openorca_result for verification.


In [5]:
from pyrecdp.LLM import TextPipeline, ResumableTextPipeline
from pyrecdp.primitives.operations import *
import os

dolly_pipeline = ResumableTextPipeline()
dolly_pipeline.enable_statistics()
out_dir = "ResumableTextPipeline_output_dolly_result"
ops = [
    ParquetReader("/content/test_data/dolly_sample_50.parquet"),
    TextPrompt(dataset_name="dolly", prompt_name="causal_llm_1"),
    RandomSelect(fraction=0.3),
    # TextToxicity(huggingface_config_path="/root/.cache/huggingface/hub/models--xlm-roberta-base"),
    TextDiversityIndicate(out_dir=out_dir, language="en", first_sent=False),
    TextQualityScorer(model="gpt3"),
    RougeScoreDedup(max_ratio=0.7, batch_size=10,score_store_path=os.path.join(out_dir,'RougeScorefiltered.parquet')),
    ParquetWriter(out_dir)
]
dolly_pipeline.add_operations(ops)
ret = dolly_pipeline.execute()
del dolly_pipeline

2023-12-05 00:13:50.408 | WARNING  | pyrecdp.LLM.TextPipeline:enable_statistics:222 - Enabling this option will result in a decrease in execution speed
[DatasetReader, PerfileSourcedParquetReader, TextPrompt, RandomSelect, TextDiversityIndicate, TextQualityScorer, RougeScoreDedup, PerfileParquetWriter]
Will assign 36 cores and 206263 M memory for spark
per core memory size is 5.595 GB and shuffle_disk maximum capacity is 8589934592.000 GB
execute with spark for global tasks started ...
DatasetReader
2023-12-05 00:13:50.953 | INFO     | pyrecdp.LLM.TextPipeline:op_summary:288 - DatasetReader: A total of 0 rows of data were processed, using 0 seconds, with 0 rows modified or removed, 0 rows of data remaining.
execute with spark for global tasks took 0.00310460664331913 sec
PerfileSourcedParquetReader


ResumableTextPipeline, current on dolly_sample_50.parquet:   0%|          | 0/1 [00:00<?, ?it/s]

dolly_sample_50.parquet
TextPrompt


RandomSelect
TextDiversityIndicate
statistics_decorator spark


/opt/conda/envs/chatbot-finetuning/lib/python3.9/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


...


0it [00:00, ?it/s]

generate_connected_components => duplicates took 0.014340104535222054 sec


2023-12-05 00:14:18.406 | INFO     | pyrecdp.LLM.TextPipeline:op_summary:288 - TextPrompt: A total of 0 rows of data were processed, using 0 seconds, with 0 rows modified or removed, 0 rows of data remaining.
2023-12-05 00:14:18.410 | INFO     | pyrecdp.LLM.TextPipeline:op_summary:288 - RandomSelect: A total of 0 rows of data were processed, using 0 seconds, with 0 rows modified or removed, 0 rows of data remaining.
2023-12-05 00:14:18.419 | INFO     | pyrecdp.LLM.TextPipeline:op_summary:284 - TextDiversityIndicate: A total of 11 rows of data were processed, using 14.846248388290405 seconds, Get max diversity types 11, Get average diversity types 1.7692307692307692,Get the std of diversity types 2.7735009811261455
2023-12-05 00:14:18.421 | INFO     | pyrecdp.LLM.TextPipeline:op_summary:284 - TextQualityScorer: A total of 11 rows of data were processed, using 1.127403974533081 seconds, Get average quality score 0.9174850873367049
2023-12-05 00:14:18.425 | INFO     | pyrecdp.LLM.TextPipe

ResumableTextPipeline, current on dolly_sample_50.parquet: 100%|██████████| 1/1 [00:27<00:00, 27.20s/it]

2023-12-05 00:14:18.432 | INFO     | pyrecdp.LLM.TextPipeline:execute:429 - Completed! ResumableTextPipeline will not return dataset, please check ResumableTextPipeline_output_dolly_result for verification.


### 3.2 Check the result and score

In [10]:
import json
import pandas as pd
import glob
print('============ Alpaca Result ============')
out_dir = 'ResumableTextPipeline_output_alpaca_result'
print('The Rouge score: ', json.load(open(f'{out_dir}/RougeScoreDedup-statistics', 'r')))
print('The toxicity: ', json.load(open(f'{out_dir}/TextToxicity-statistics', 'r')))
print('The diversity intricate: ', json.load(open(f'{out_dir}/TextDiversityIndicate-statistics', 'r')))
print('The quality score :', json.load(open(f'{out_dir}/TextQualityScorer-statistics', 'r')))


origin_dataset_length = (len(pd.read_json("/content/test_data/alpaca_data_50.jsonl", lines=True))) 
downsized_dataset_length = len(pd.read_parquet(glob.glob(f'{out_dir}/alpaca_data_50.jsonl/*.parquet')))

print(f'The original dataset length: {origin_dataset_length}, the processed dataset length: {downsized_dataset_length}')

print('============ Dolly Result ============')
out_dir = 'ResumableTextPipeline_output_dolly_result'
print('The Rouge score: ', json.load(open(f'{out_dir}/RougeScoreDedup-statistics', 'r')))
# print('The toxicity: ', json.load(open(f'{out_dir}/TextToxicity-statistics', 'r')))
print('The diversity intricate: ', json.load(open(f'{out_dir}/TextDiversityIndicate-statistics', 'r')))
print('The quality score :', json.load(open(f'{out_dir}/TextQualityScorer-statistics', 'r')))


origin_dataset_length = (len(pd.read_parquet("/content/test_data/dolly_sample_50.parquet"))) 
downsized_dataset_length = len(pd.read_parquet(glob.glob(f'{out_dir}/dolly_sample_50.parquet/*.parquet')))

print(f'The original dataset length: {origin_dataset_length}, the processed dataset length: {downsized_dataset_length}')

print('============ Openorca Result ============')
out_dir = 'ResumableTextPipeline_output_openorca_result'
print('The Rouge score: ', json.load(open(f'{out_dir}/RougeScoreDedup-statistics', 'r')))
# print('The toxicity: ', json.load(open(f'{out_dir}/TextToxicity-statistics', 'r')))
print('The diversity intricate: ', json.load(open(f'{out_dir}/TextDiversityIndicate-statistics', 'r')))
print('The quality score :', json.load(open(f'{out_dir}/TextQualityScorer-statistics', 'r')))


origin_dataset_length = (len(pd.read_parquet("/content/test_data/openorca_sample_50.parquet"))) 
downsized_dataset_length = len(pd.read_parquet(glob.glob(f'{out_dir}/openorca_sample_50.parquet/*.parquet')))

print(f'The original dataset length: {origin_dataset_length}, the processed dataset length: {downsized_dataset_length}')

============ Alpaca Result ============
The Rouge score:  {'dup_num': 0, 'dup_ratio': 0.0}
The toxicity:  {'min': 0.0001716656406642869, 'max': 0.0010136101627722383, 'mean': 0.00036636507800060576, 'std': 0.0002271174606949808}
The diversity intricate:  {'max': 11, 'mean': 1.625, 'std': 2.5}
The quality score : {'mean': 0.9509534303108631}
The original dataset length: 50, the processed dataset length: 11
============ Dolly Result ============
The Rouge score:  {'dup_num': 0, 'dup_ratio': 0.0}
The diversity intricate:  {'max': 11, 'mean': 1.7692307692307692, 'std': 2.7735009811261455}
The quality score : {'mean': 0.9174850873367049}
The original dataset length: 50, the processed dataset length: 11
============ Openorca Result ============
The Rouge score:  {'dup_num': 0, 'dup_ratio': 0.0}
The diversity intricate:  {'max': 11, 'mean': 1.826086956521739, 'std': 2.328717436956612}
The quality score : {'mean': 0.9467587080940774}
The original dataset length: 50, the processed dataset lengt